In [2]:
import pandas as pd
import numpy as np
import os

# Pull Data

In [3]:
# create train and test indices
processed_data_dir = 'C:/Users/Gaurav Anand/Downloads/'
train = pd.read_parquet(os.path.join(processed_data_dir, 'HSP-L_Nowplay_RS_Train.parquet'))
test  = pd.read_parquet(os.path.join(processed_data_dir, 'HSP-L_Nowplay_RS_Test.parquet'))

In [4]:
train 

,user id,source of the tweet,timestamp,track title,artist name,musicbrainz identifier of the recording,highlevel.danceability.value,highlevel.genre_dortmund.value,highlevel.ismir04_rhythm.value,highlevel.mood_acoustic.value,...,instrumentalness,danceability,valence,loudness,tempo,acousticness,energy,mode,key,rating
0,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-01-01 09:53:50,Yellow,Coldplay,f5df0496-2fd5-4601-b59b-1f8154d81e55,not_danceable,rock,VienneseWaltz,not_acoustic,...,0.00013,0.429,0.279,-7.227,173.365,0.00239,0.661,1,11,1
1,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-01-01 15:58:20,Again,Lenny Kravitz,942c1e62-a7d3-4340-833c-65430fc13c75,not_danceable,blues,VienneseWaltz,not_acoustic,...,4.91e-05,0.554,0.767,-5.035,79.155,0.0186,0.82,1,2,1
2,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-01-01 20:44:30,Mama,Spice Girls,3f851b4f-51cf-4d09-b136-32605f406492,not_danceable,folkcountry,Rumba-International,not_acoustic,...,0.543,0.615,0.801,-9.328,95.848,0.345,0.734,1,8,1
3,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-01-01 22:06:09,Here Without You,3 Doors Down,9d15aaee-64fa-4f6e-9962-3b1418376a7e,not_danceable,rock,VienneseWaltz,not_acoustic,...,0,0.551,0.224,-6.765,144.001,0.0587,0.543,0,10,1
4,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-01-01 22:55:02,Clocks,Coldplay,c3e77d6a-700b-4a8e-bdc7-c8a53c3f6fbf,not_danceable,alternative,VienneseWaltz,acoustic,...,0.0112,0.577,0.271,-7.215,130.969,0.599,0.749,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,328cfb62152aa1df185b7bf90283b493d1f34449,Now Playing Tool for OtsAV,2014-05-25 01:14:30,No Limit,2 Unlimited,e472c97a-98a8-4d89-b139-1ac422918946,danceable,rock,ChaChaCha,not_acoustic,...,0,0.711,0.71,-10.646,91.088,0.107,0.53,0,4,1
2313,dc0b3a6b4a94031afe916da2b6d03fb93225f62c,Twitter Web Client,2014-06-09 15:53:40,Finally Forever,Chris Cornell,273549c1-df0e-44e7-b8da-e589f536f441,danceable,blues,ChaChaCha,not_acoustic,...,0.861,0.546,0.425,-4.757,119.991,1.74e-05,0.945,0,1,1
2314,527d5f12272c34b5aae8e18770b200c25ffcb225,Twitter Web Client,2014-06-16 15:52:41,Clocks,Coldplay,c3e77d6a-700b-4a8e-bdc7-c8a53c3f6fbf,not_danceable,alternative,VienneseWaltz,acoustic,...,0.0112,0.577,0.271,-7.215,130.969,0.599,0.749,0,5,1
2315,9b1ce39062e55428b72195c1938f6dfa745f0756,WZBTPlaylist Script,2014-07-06 08:22:02,Emerge,Fischerspooner,539a192b-fa8d-42e2-b38c-80c63fcc43f9,danceable,electronic,ChaChaCha,not_acoustic,...,0.0204,0.719,0.428,-6.340,127.963,0.00201,0.847,0,0,1


## Pull lyrics for train and test data from Lyrics Genius API

In [5]:
import pandas as pd
import lyricsgenius
import re

# Initialize Genius API
API_KEY = "vCzYfIrd5rG7XGeLBH0H6dDjHo9U4LE5IXLcumb_ELVh5QreL7Ug8ThUbmpnbGaL" # my genius api key, we can all use this.
genius = lyricsgenius.Genius(API_KEY)


# Remove anything that's not a letter, number, underscore, or whitespace. Also remove's punctuation and special symbols that might affect the match rate with Genius. 
def preprocess_string(s):
    return re.sub(r'[^\w\s]', '', s).strip().lower()

def fetch_lyrics(artist, track_title):
    try:
        song = genius.search_song(title=track_title, artist=artist)
        
        if song:
            fetched_artist = preprocess_string(song.artist)
            fetched_title = preprocess_string(song.title)
            input_artist = preprocess_string(artist)
            input_title = preprocess_string(track_title)
            
            if input_artist in fetched_artist and input_title in fetched_title:
                return song.lyrics
            else:
                print(f"No exact match for: {artist} - {track_title}")
                return None
        else:
            print(f"Song not found: {artist} - {track_title}")
            return None
    except Exception as e:
        print(f"Error fetching lyrics for {artist} - {track_title}: {e}")
        return None

# Apply the fetch_lyrics function to each row in the dataset
lyrics_list = []
for index, row in train.iterrows():
    artist = row['Artist']
    track_title = row['Title']
    lyrics = fetch_lyrics(artist, track_title)
    lyrics_list.append(lyrics)


print("Lyrics fetching complete.")
    
      


Searching for "Yellow" by Coldplay...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Clocks" by Coldplay...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "More Love" by Kim Carnes...
Done.
Searching for "Yellow" by Coldplay...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Clocks" by Coldplay...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Clocks" by Coldplay...
Done.
Searching for "Parklife" by Blur...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Yellow" by Coldplay...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Mama" by Spice Girls...
Do

In [6]:
train['lyrics'] = lyrics_list

In [7]:
import pandas as pd
import lyricsgenius
import re

# Preprocess strings to improve matching
def preprocess_string(s):
    return re.sub(r'[^\w\s]', '', s).strip().lower()

# Function to search for a song on Genius and pull lyrics
def fetch_lyrics(artist, track_title):
    try:
        # Search for the song on Genius
        song = genius.search_song(title=track_title, artist=artist)
        
        if song:
            # Preprocess and match artist and title for accuracy
            fetched_artist = preprocess_string(song.artist)
            fetched_title = preprocess_string(song.title)
            input_artist = preprocess_string(artist)
            input_title = preprocess_string(track_title)
            
            if input_artist in fetched_artist and input_title in fetched_title:
                return song.lyrics
            else:
                print(f"No exact match for: {artist} - {track_title}")
                return None
        else:
            print(f"Song not found: {artist} - {track_title}")
            return None
    except Exception as e:
        print(f"Error fetching lyrics for {artist} - {track_title}: {e}")
        return None

# Apply the fetch_lyrics function to each row in the dataset
lyrics_list = []
for index, row in test.iterrows():
    artist = row['Artist']
    track_title = row['Title']
    lyrics = fetch_lyrics(artist, track_title)
    lyrics_list.append(lyrics)


print("Lyrics fetching complete.")

test['lyrics'] = lyrics_list
      


Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Wander This World" by Jonny Lang...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Make Me Believe" by Godsmack...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "New York Groove" by Ace Frehley...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Cold Hearted" by Paula Abdul...
Done.
Searching for "Mama" by Spice Girls...
Done.
Searching for "Breakpoint" by Megadeth...
Done.
Searching for "Here Without You" by 3 Doors Down...
Done.
Searching for "Again" by Lenny Kravitz...
Done.
Searching for "Blue" by Lucinda Williams...
Done.
Searching for "Mama" by Spice G

In [31]:
train['lyrics'] = train['lyrics'].apply(lambda x: x[x.lower().find('lyrics') + len('lyrics'):].strip() if isinstance(x, str) and 'lyrics' in x.lower() else x)
test['lyrics'] = test['lyrics'].apply(lambda x: x[x.lower().find('lyrics') + len('lyrics'):].strip() if isinstance(x, str) and 'lyrics' in x.lower() else x)

In [32]:
train['lyrics']

0       [Verse 1: Chris Martin]\nLook at the stars\nLo...
1       [Verse 1]\nI've been searching for you\nI hear...
2       [Verse 1: Baby with Sporty]\nShe used to be my...
3       [Verse 1]\nA hundred days have made me older\n...
4       [Verse 1]\nThe lights go out, and I can't be s...
                              ...                        
2312    [Intro: Ray Slijngaard]\nLet me hear you say y...
2313    [Verse 1]\nOn the silent night so out of place...
2314    [Verse 1]\nThe lights go out, and I can't be s...
2315    Hi\nHuh-i\nHyper\nHyper-media-ocrity.....\n\nY...
2316    If you turn me down once more, I'll join the F...
Name: lyrics, Length: 2317, dtype: object

In [12]:
len(train['Title'].unique())

349

In [14]:
len(train['Artist'].unique())

256

In [63]:
import torch
from transformers import RobertaTokenizer, RobertaModel
import nltk
from nltk.tokenize import sent_tokenize


# Load pre-trained RoBERTa model and tokenizer
# Load pre-trained RoBERTa model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base').to(device)

# Function to generate embeddings using mean pooling
def generate_roberta_embedding(text):
    if isinstance(text, str):
        # Tokenize input text
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding=True).to(device)
        # Pass through RoBERTa model
        with torch.no_grad():
            outputs = model(**inputs)
        # Get the last hidden states
        last_hidden_state = outputs.last_hidden_state
        # Apply mean pooling
        mean_embedding = torch.mean(last_hidden_state, dim=1).squeeze()
        return mean_embedding
    else:
        return None

# Generate embeddings for all lyrics and store them
embeddings = []
for lyrics in train['lyrics']:
    embedding = generate_roberta_embedding(lyrics)
    if embedding is not None:
        embeddings.append(embedding.cpu().numpy())
    else:
        embeddings.append(None)

# Add the embeddings to the dataframe
train['lyrics_embedding'] = embeddings

# Generate embeddings for all lyrics and store them
embeddings = []
for lyrics in test['lyrics']:
    embedding = generate_roberta_embedding(lyrics)
    if embedding is not None:
        embeddings.append(embedding.cpu().numpy())
    else:
        embeddings.append(None)

# Add the embeddings to the dataframe
test['lyrics_embedding'] = embeddings


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [77]:
display(A := train.loc[0, ['Title', 'Artist', 'lyrics_embedding']], (B:= train.loc[1003, ['Title', 'Artist', 'lyrics_embedding']]))

def cosine_similarity(A,B):
        
    dot_product = np.dot(A, B)
    magnitude_a = np.linalg.norm(A)
    magnitude_b = np.linalg.norm(B)
    
    return dot_product / (magnitude_a * magnitude_b)


cosine_similarity(A['lyrics_embedding'], B['lyrics_embedding'])

Title                                                          Yellow
Artist                                                       Coldplay
lyrics_embedding    [-0.03243772, -0.0019709643, 0.02333304, -0.13...
Name: 0, dtype: object

Title                                                     Better Life
Artist                                                    Keith Urban
lyrics_embedding    [0.04753896, 0.048531726, 0.06382249, -0.08452...
Name: 1003, dtype: object

0.98457533

In [107]:
display(A := train.loc[0, ['Title', 'Artist', 'lyrics_embedding']], (B:= train.loc[1989, ['Title', 'Artist', 'lyrics_embedding']]))

def cosine_similarity(A,B):
        
    dot_product = np.dot(A, B)
    magnitude_a = np.linalg.norm(A)
    magnitude_b = np.linalg.norm(B)
    
    return dot_product / (magnitude_a * magnitude_b)


cosine_similarity(A['lyrics_embedding'], B['lyrics_embedding'])

Title                                                          Yellow
Artist                                                       Coldplay
lyrics_embedding    [-0.03243772, -0.0019709643, 0.02333304, -0.13...
Name: 0, dtype: object

Title               I Think
Artist              Alexkid
lyrics_embedding       None
Name: 1989, dtype: object

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [110]:
train.to_parquet(os.path.join(processed_data_dir, 'EmbeddedLyrics_HSP-L_Nowplay_RS_Train.parquet'))

In [111]:
test.to_parquet(os.path.join(processed_data_dir, 'EmbeddedLyrics_HSP-L_Nowplay_RS_Test.parquet'))

In [61]:
train.iloc[7]

user id                                             96b828adf6079cac757e2bbfc59fdd81d62e6996
source of the tweet                                               SG Radio Now Playing Tweet
timestamp                                                                2014-01-02 23:40:00
track title                                                                           Yellow
artist name                                                                         Coldplay
musicbrainz identifier of the recording                 f5df0496-2fd5-4601-b59b-1f8154d81e55
highlevel.danceability.value                                                   not_danceable
highlevel.genre_dortmund.value                                                          rock
highlevel.ismir04_rhythm.value                                                 VienneseWaltz
highlevel.mood_acoustic.value                                                   not_acoustic
highlevel.mood_happy.value                                            

In [109]:
test.head()

,user id,source of the tweet,timestamp,track title,artist name,musicbrainz identifier of the recording,highlevel.danceability.value,highlevel.genre_dortmund.value,highlevel.ismir04_rhythm.value,highlevel.mood_acoustic.value,...,valence,loudness,tempo,acousticness,energy,mode,key,rating,lyrics,lyrics_embedding
0,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-10-01 20:43:45,Again,Lenny Kravitz,942c1e62-a7d3-4340-833c-65430fc13c75,not_danceable,blues,VienneseWaltz,not_acoustic,...,0.767,-5.035,79.155,0.0186,0.82,1,2,1,[Verse 1]\nI've been searching for you\nI hear...,"[0.040212587, 0.07172622, -0.022398397, -0.148..."
1,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-10-02 01:43:04,Here Without You,3 Doors Down,9d15aaee-64fa-4f6e-9962-3b1418376a7e,not_danceable,rock,VienneseWaltz,not_acoustic,...,0.224,-6.765,144.001,0.0587,0.543,0,10,1,[Verse 1]\nA hundred days have made me older\n...,"[-0.019110568, 0.087903485, 0.04418295, -0.132..."
2,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-10-02 05:52:03,Mama,Spice Girls,3f851b4f-51cf-4d09-b136-32605f406492,not_danceable,folkcountry,Rumba-International,not_acoustic,...,0.801,-9.328,95.848,0.345,0.734,1,8,1,[Verse 1: Baby with Sporty]\nShe used to be my...,"[-0.02841081, 0.11925947, 0.019223021, -0.0387..."
3,96b828adf6079cac757e2bbfc59fdd81d62e6996,SG Radio Now Playing Tweet,2014-10-03 00:09:39,Again,Lenny Kravitz,942c1e62-a7d3-4340-833c-65430fc13c75,not_danceable,blues,VienneseWaltz,not_acoustic,...,0.767,-5.035,79.155,0.0186,0.82,1,2,1,[Verse 1]\nI've been searching for you\nI hear...,"[0.040212587, 0.07172622, -0.022398397, -0.148..."
4,25ecdc1a1cb85938b41c633797c093312a7834da,RadioTwitSoftware,2014-10-03 02:35:08,Wander This World,Jonny Lang,053996f6-70ed-41ad-8eb5-f3137377bd58,not_danceable,blues,ChaChaCha,not_acoustic,...,0.716,-3.769,119.002,0.00264,0.919,1,0,1,The tar in the street starts to melt from the ...,"[-0.018534057, 0.035135806, 0.03772033, -0.070..."


In [116]:
train.iloc[5]['lyrics_embedding'].shape

(768,)

In [127]:
train.value_counts(['Artist','Title']).reset_index(drop=False).loc[0:50, 'count'].sum()

1737

In [121]:
train.shape[0]

2317

In [123]:
sum([155,137,137,113,110,101])


753

In [128]:
1737/2317

0.7496763055675443

In [ ]:
!pip install zenodo_get